# MuMoT Demonstration Notebook: Agent Density in Collective Decisions <a class="tocSkip">

## Multiscale Modelling Tool <a class="tocSkip">

*Yara Khaluf, Department of Information Technology, Ghent University;
Andreagiovanni Reina, Department of Computer Science, University of Sheffield;
Thomas Bose, Department of Computer Science, University of Sheffield;
James A. R. Marshall, Department of Computer Science, University of Sheffield*

# Introduction

This notebook reproduces and extends the results of Khaluf *et al.* ([2017](#references)). In that paper, based on an earlier analysis by Biancalini *et al.* ([2014](#references)), the authors show that non-uniform distributions in swarms of agents have an impact on the scalability of collective decision-making. In particular, they highlight the relevance of noise-induced bistability in very sparse swarm systems and the failure of these systems to scale. Their work is based on three decision models: In the first model, each agent can change its decision after being recruited by a nearby agent. The second model captures the dynamics of dense swarms controlled by the majority rule (i.e., agents switch their opinion to comply with that of the majority of their neighbours). The third model combines the first two, with the aim of studying the role of non-uniform swarm density in the performance ofcollective decision-making. Based on the three models, the authors formulate a set of requirements for convergence and scalability in collective decision-making.



In [ ]:
import mumot
mumot.about()

## Defining the spontaneous-switching model
The model presented in the next cell consists of two parts: (i) random recruitment with a reaction rate $r$, (ii) and spontaneous switch with rate $\epsilon$.

In [ ]:
%%model
$
B -> A : \epsilon
A -> B : \epsilon
A + B -> A + A : r
B + A -> B + B : r
$

In [ ]:
model1 = mumot.parseModel(In[-2])

In [ ]:
model1.show()

In [ ]:
model1.visualise()

# Results
## Noise-free stability in infinite populations
The model just presented only exhibits a single stable fixed point, regardless of parameterisation, as can be seen by analysis of the ODE system describing its infinite population, mean-field behaviour.

In [ ]:
model1.showODEs()

Setting $dA/dt=dB/dt=0$ for non-zero $\epsilon$ is only possible when $A$ = $B$, and this fixed point can be shown to be stable (Biancalini *et al.* [2014](#references)).

## Noise-induced bistability in a well-mixed system
In contrast to its noise-free dynamics, the model displays noise-induced bistability. Depending on the rate coefficient $\epsilon$, a critical system size $N_c=1/\epsilon$ exists, such that the system is bistable for $N < N_c$ (*i.e.* bimodal stationary distributions) and unistable for $N > N_c$ (*i.e.* unimodal stationary distributions). 

You can experiment with the effect of varying $\epsilon$ in the following simple multicontroller. The lefthand view is for system size $N=50$ whereas the righthand view is for $N=200$. Recall that the critical systemsize is $N_c=1/\epsilon$, which for the initial value of $\epsilon=0.01$ in the controller gives $N_c=100$.

In [ ]:
mc1 = mumot.MuMoTmultiController([model1.SSA(params = [('systemSize', 50)], silent = True),
                                  model1.SSA(params = [('systemSize', 200)], silent = True)],
                                 shareAxes = False, params = [('r', 1.0), ('plotLimits', 1)],
                                initialState = {'A': 0.5, 'B': 0.5}, maxTime = 100.0, randomSeed = 112783855, 
                                plotProportions = True, initWidgets = {'epsilon':[0.01,0,0.1,0.005]},
                                realtimePlot = False, aggregateResults = False, runs=1, visualisationType = 'evo')


The previous controller showed results from single simulation runs. We can view the distribution of system states after a fixed simulation duration with the following multicontroller. Again, the lefthand view is for system size $N=50$ whereas the righthand view is for $N=200$.

In [ ]:
mc2 = mumot.MuMoTmultiController([model1.SSA(params = [('systemSize', 20)], silent = True),
                                  model1.SSA(params = [('systemSize', 100)], silent = True)],
                                 shareAxes = False, params = [('r', 1.0), ('plotLimits', 1)],
                                initialState = {'A': 0.5, 'B': 0.5}, maxTime = 10, randomSeed = 112783856, 
                                plotProportions = True, initWidgets = {'epsilon':[0.01,0,0.1,0.005]},
                                realtimePlot = False, aggregateResults = False, runs=30,
                                visualisationType = 'final', final_x = 'A', final_y = 'B')


## Effects of space on system dynamics

The next cell shows the impact of the interaction network on the system dynamics. With $\epsilon=0.01$ and a system size of S=40 reactants, the system is parameterised under the critical threshold predicting symmetry breaking (S<100).

By changing the interaction range, we can obtain symmetry breaking or deadlock. If the interaction range is small (try 0.05), the noise dominates the dynamics (equivalent to large $\epsilon$), conversely when the interaction range is large (try 0.5) the influence of noise is counteracted by recruitment (rate $r$) and the dynamics are similar to the mean-field analyses illustrated above. Similarly, the same effect can be observed with other network types (*e.g.* try a Barabasi-Albert network and vary the connectivity parameter `new edges`).

In [ ]:
ma1 = model1.multiagent(initWidgets = {'\epsilon':[0.01,0,0.1,0.005], 'maxTime':[100,5,200,5], 'netType':'dynamic',
                                      'systemSize':[40,10,200,1], 'showInteractions':True},
                       initialState = {'A': 0.5, 'B': 0.5}, params = [('r', 1.0)], randomSeed=261594811,
                       timestepSize=0.99, motionCorrelatedness = 0.6, particleSpeed = 0.01, plotProportions = True)

## Relationship to models of honeybee house-hunting
### Approximation for indirect switching
The above model (Biancalini *et al.* [2014](#references), Khaluf *et al.*, [2017](#references)) is similar to the *Apis mellifera* model *House-hunting with indirect switching* of Marshall *et al.* ([2009](#references)). The latter model is defined below for the symmetric, equal alternatives, case and has the additional state $U$ representing the uncommitted bees. The model has three main transitions: discovery from $U$ to $A$ (or $B$) at rate $q$; decay from $A$ (or $B$) to $U$ at rate $k$; and recruitment of $U$ at rate $g$.

In this case, we can approximate the spontaneous switch $\epsilon$ (from Khaluf *et al.* ([2017](#references))) as the combination of the transitions of decay $A \xrightarrow{k} U$ and discovery $U  \xrightarrow{q} B$ (or $A$), i.e. 
$A \xrightarrow{k} U \xrightarrow{\frac{q}{2}} B$
Therefore, we approximate $\epsilon \approx \frac{k\,q}{2}$.
Similarly, the recuritment $r$ from Khaluf *et al.* ([2017](#references)) can be approximated as $r \approx k\,g$.

In [ ]:
%%model
$
U -> A : q
U -> B : q
A -> U : k
B -> U : k
U + A -> A + A : g
U + B -> B + B : g
$

In [ ]:
model2 = mumot.parseModel(In[-2]).substitute('U=N-A-B')

In [ ]:
model2.show()

In [ ]:
model2.visualise()

For this model we can first explore our intuition that there is only one stable attractor in the infinite population case.

In [ ]:
str1 = model2.stream('A','B', showFixedPoints = True)

Next, using the above approximation, we set the following parameters $k=0.01$, $g=100$, and $q=1$. This leads to 
$r \approx k\,g=1$ and $\epsilon \approx \frac{k\,q}{2} = 0.005$. The critical system size is thus $N_c \approx 200$. In the following multicontroller the lefthand view is for system size $N=50$, and the righthand view is for $N=300$.

In [ ]:
mc3 = mumot.MuMoTmultiController([model2.SSA(params = [('systemSize', 50)], silent = True),
                                  model2.SSA(params = [('systemSize', 300)], silent = True)],
                                 shareAxes = False, params = [('g', 100.0), ('plotLimits', 1)],
                                initialState = {'A': 0.1, 'B': 0.1, 'U': 0.8}, maxTime = 2000.0, randomSeed = 112783855, 
                                plotProportions = True, initWidgets = {'k':[0.01,0,0.1,0.005],'q':[1,0,1,0.05]},
                                realtimePlot = False, aggregateResults = False, runs=1, visualisationType = 'evo')


### Equivalence for direct switching with spontaneous switches

The preceding analysis was an approximation to a model of indirect switching. However, Marshall *et al.* ([2009](#references)) also analysed a *direct switching* model and argued that this decision-making model is statistically optiml for decay $k=0$. Yet decision-making in this model is pathological in the case where two equal alternatives are under consideration, and decay is zero; here a means of breaking symmetry is required (Seeley *et al.*, [2012](#references)). In fact, when $k=0$ then the uncommitted population $U$ will approach 0, and in this limit the symmetric honeybee house-hunting model is identical to the model considered by Khaluf *et al.* ([2017](#references)) but with $\epsilon=0$. Thus noise-induced bistability should be able to break deadlock in the model of Marshall *et al.* ([2009](#references)) by adding additional transitions, as can be seen below.

In [ ]:
%%model
$
U -> A : q
U -> B : q
A -> B : \epsilon
B -> A : \epsilon
U + A -> A + A : g
U + B -> B + B : g
A + B -> A + A : r
A + B -> B + B : r
$

Again, we explore the infinite population model, seeing that there is only ever a single stable attractor.

In [ ]:
model3 = mumot.parseModel(In[-2]).substitute('U=N-A-B')
model3.show()

In [ ]:
model3.visualise()

In [ ]:
str2 = model3.stream('A','B', showFixedPoints = True, params = [('r', 1.0)])

Next, we can make use of the exact same calculation of the critical system size $N_c=1/\epsilon$ used above to determine when noise-induced bistability should be observed.

In [ ]:
mc4 = mumot.MuMoTmultiController([model3.SSA(params = [('systemSize', 50)], silent = True),
                                  model3.SSA(params = [('systemSize', 200)], silent = True)],
                                  shareAxes = False, params = [('g', 1.0), ('q', 1.0), ('r', 1.0), ('plotLimits', 1)],
                                  initialState = {'A': 0.1, 'B': 0.1, 'U': 0.8}, initWidgets = {'epsilon':[0.01,0,0.1,0.005]},
                                  maxTime = 100.0, randomSeed = 112783855, plotProportions = True, realtimePlot = False,
                                  aggregateResults = False, runs=1, visualisationType = 'evo')



# Discussion

The results presented above have shown how noise-induced bistability can break decision deadlocks. However, introducing spontaneous switching is not a silver bullet. While spontaneous switching can break deadlock in the case of equal decison alternatives, it can also make decision-making suboptimal when alternatives differ. Optimality in the direct-switching model analysed by Marshall *et al.* can be shown by reducing the dynamics of the model when $U=0$ to the statistically-optimal Drift-Diffusion Model (DDM) of decision-making (Marshall *et al.* [2009](#references))
$$
\dot{x}=A+\sigma \eta
$$
where $\dot{x}$ is the change in evidence variable $x$, $A$ is constant drift proportional to decision signal, and $\sigma \eta$ is Gaussian noise proportional to decision noise.

Introducing spontaneous switches it is easy to show, using the method presented in Marshall *et al.* ([2009](#references)), that the decision process when $U=0$ becomes
$$
\dot{x}=A+ B(x + \sigma' \eta) + \sigma \eta
$$
where $B<0$ is a stabilising term an $\sigma'$ is a second noise process, making the decision process akin to a generalised stable Ornstein-Uhlenbeck process, which must be suboptimal. In contrast, the *stop-signalling* model of house-hunting in honeybees (Seeley *et al.*, [2012](#references)) is able both to break decision deadlock in the case of equal decision alternatives, and approximate the statistically optimal DDM when alternatives differ (Pais *et al.*, [2013](#references)).

# References 
<a id='references'></a>

* Biancalani, T., Dyson, L., McKane, A.J., (2014) [Noise-induced bistable states and their mean switching time in foraging colonies](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.112.038101). *Physical Review Letters* **112**(3), 038101.
* Khaluf, Y., Pinciroli, C., Valentini, G., Hamann, H. (2017) [The impact of agent density on scalability in collective systems: noise-induced versus majority-based bistability](https://doi.org/10.1007/s11721-017-0137-6). *Swarm Intelligence*  **11**: 155-179. 
* Marshall, J. A. R., Bogacz, R., Dornhaus, A. Planqué, R., Kovacs, T. & Franks, N. R. (2009) [On optimal decision making in brains and social insect colonies](https://doi.org/10.1098/rsif.2008.0511). *Journal of the Royal Society: Interface* **6**, 1065-1074.
* Pais, D., Hogan, P.M., Schlegel, T., Franks, N.R., Leonard, N.E. & Marshall, J.A.R. (2013) [A mechanism for value-sensitive decision-making](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0073216).  *PLoS one* **8**(9), e73216
* Seeley, T.D, Visscher, P.K. Schlegel, T., Hogan, P.M., Franks, N.R. & Marshall, J.A.R. (2012) [Stop signals provide cross inhibition in collective decision-making by honeybee swarms](http://www.sciencemag.org/content/335/6064/108.full.pdf). *Science* **335**, 108-111